# Classify Images of Flowers
**Purpose**:  
Use transfer learning to classify images of cats and dogs using [TensorFlow Hub](https://www.tensorflow.org/hub), MobileNet models and the Dogs vs. Cats dataset.

Concepts Covered:
1. Use a TensorFlow Hub model for prediction
2. Use a TensorFlow Hub model for Dogs vs. Cats dataset
3. Do simple transfer learning with TensorFlow Hub

Dataset Used: [Dogs vs Cats dataset](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs)


Project based on [TensorFlow's transfer learning example](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l06c01_tensorflow_hub_and_transfer_learning.ipynb)


In [ ]:
# import tf and dependencies
import tensorflow as tf
import matplotlib.pylab as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
# download the classifier
CLASSIFIER_URL ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
IMAGE_RES = 224

model = tf.keras.Sequential([
    hub.KerasLayer(CLASSIFIER_URL, input_shape=(IMAGE_RES, IMAGE_RES, 3))
])

**Use a TensorFlow Hub MobileNet for prediction**  
Take a trained model and load it into Keras
1. Download the classifier
2. Run on a single image
3. Decode the predictions

In [ ]:
# test if the model can work on a different output class
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize((IMAGE_RES, IMAGE_RES))
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

In [ ]:
# add a batch dimension and pass the image to the model for prediction
result = model.predict(grace_hopper[np.newaxis, ...])
result.shape

In [ ]:
# ask for predicted class
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

In [ ]:
# decode the predictions
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

**Use a TensorFlow Hub model for the Dogs vs. Cats datset**  
Use the full MobileNet model and see how it performs on the Dogs vs. Cats dataset
1. Use TF Datasets to load the dataset
2. Reformat images
3. Run the classifier on a batch of images

In [ ]:
# load Dogs vs. Cats dataset
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    with_info=True,
    as_supervised=True,
    split=['train[:80%]', 'train[80%:]']
)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
# check the size of the images in the dataset
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches      = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
# reformat images to the resolution expected by MobileNet
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches      = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
# run the classifier on a batch of images
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_bacth = label_batch.numpy()

result_batch = model.predict(image_batch)

predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

In [ ]:
# plot function
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')

_ = plt.suptitle("ImageNet predictions")

**Do simple transfer learning with TensorFlow Hub**  
Reuse parts of an already trained model and alter it to work with our own dataset.

1. Extract and freeze the trained model
2. Attach a classification head
3. Train the model
4. Check the predictions

In [ ]:
# extract the dataset
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [ ]:
# run a batch of images through to see final shape (1290 neurons)
feature_batch = feature_extractor(image_batch)
print(feature_batch.shape)

In [ ]:
# freeze the variables in the feature extractor layer
feature_extractor.trainable = False

In [ ]:
# attach a classification head by wrapping the hub layer in a Sequential model
# and adding a new classification layer

model = tf.keras.Sequential([
                             feature_extractor,
                             layers.Dense(2)
])

model.summary()

In [ ]:
# train the model
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
EPOCHS = 6
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [ ]:
# plot the training and validation accuracy/loss graphs using matplot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

# if validation performance > training performance, it's ok!
# since we're reusing parts of the MobileNet dataset, it already includes 
# dog and cat images    
# also, augmentation is only applied to the training images, not the
# validation set ==> training images might be harder to classify than
# images in the validation set

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# check predictions and redo the previous plot

# get the ordered list of class names
class_names = np.array(info.features['label'].names)
class_names

In [ ]:
# run the image batch through the model, convert indices to class names
predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

In [ ]:
# look at true and predicted labels
print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)

In [ ]:
# plot
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")